<a href="https://colab.research.google.com/github/lhyochan7/MRI-analysis/blob/main/MRI_pix2pix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install patchify
import nibabel as nib
import tensorflow as tf
import numpy as np
import os
import glob
import time
import datetime
from nibabel.viewers import OrthoSlicer3D


from patchify import patchify, unpatchify
from matplotlib import pyplot as plt
from numpy import load
from numpy import zeros
from numpy import ones
from numpy.random import randint
from keras.optimizers import Adam
from keras.initializers import RandomNormal
from keras.models import Model
from keras.models import Input
from keras.layers import Conv3D
from keras.layers import Conv3DTranspose
from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.layers import Concatenate
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from matplotlib import pyplot
from numpy import savez_compressed


In [2]:
'''
class GAN():
  def __init__(self):
    self.height = 64
    self.width = 64
    self.depth = 64
    self.channel = 1
    self.img_shape = (self.height, self.width, self.depth, self.channel)

    optimizer = Adam(lr=0.002, beta_1=0.5, beta_2=0.999)

    self.discriminator = self.build_discriminator()
    self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    self.generator = self.build_generator()
    self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)

    in_src = Input(shape=self.img_shape)
    img = self.generator(in_src)

    self.discriminator.trainable = False

    valid = self.discriminator([in_src,img])
  
    self.combined = Model(in_src, [valid, img])
    self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)
    
  def downsample(self, n_filters, size, apply_batchnorm=True):
    initializer = tf.random_normal_initializer(0., 0.02)

    result = tf.keras.Sequential()
    result.add(
        tf.keras.layers.Conv3D(n_filters, size, (2,2,2), padding='same',
                              kernel_initializer=initializer, use_bias=False))
    if apply_batchnorm:
      result.add(tf.keras.layers.BatchNormalization())

    result.add(tf.keras.layers.LeakyReLU())

    return result


  def upsample(self, n_filters, size, apply_dropout=False):
    initializer = tf.random_normal_initializer(0., 0.02)

    result = tf.keras.Sequential()
    result.add(
      tf.keras.layers.Conv3DTranspose(n_filters, size, strides=(2,2,2),
                                      padding='same',
                                      kernel_initializer=initializer,
                                      use_bias=False))

    result.add(tf.keras.layers.BatchNormalization())

    if apply_dropout:
        result.add(tf.keras.layers.Dropout(0.5))

    result.add(tf.keras.layers.ReLU())

    return result

  def build_generator(self):
    inputs = tf.keras.layers.Input(shape=self.img_shape)

    down_stack = [
      self.downsample(64, 3, apply_batchnorm=False),  # (batch_size,64,64,64,1)
      self.downsample(128, 3),  # (batch_size, 64, 64, 128)
      self.downsample(256, 3),  # (batch_size, 32, 32, 256)
      self.downsample(512, 3),  # (batch_size, 16, 16, 512)
      self.downsample(512, 3),  # (batch_size, 8, 8, 512)
      self.downsample(512, 4),  # (batch_size, 4, 4, 512)
    ]

    up_stack = [
      self.upsample(512, 3, apply_dropout=True),  # (batch_size, 2, 2, 1024)
      self.upsample(512, 3, apply_dropout=True),  # (batch_size, 4, 4, 1024)
      self.upsample(256, 3),  # (batch_size, 32, 32, 512)
      self.upsample(128, 3),  # (batch_size, 64, 64, 256)
      self.upsample(64, 3),  # (batch_size, 128, 128, 128)
    ]

    initializer = tf.random_normal_initializer(0., 0.0002)
    last = tf.keras.layers.Conv3DTranspose(1, 3,
                                          strides=(2,2,2),
                                          padding='same',
                                          kernel_initializer=initializer,
                                          activation='tanh')  # (batch_size, 256, 256, 3)

    x = inputs

    # Downsampling through the model
    skips = []
    for down in down_stack:
      x = down(x)
      skips.append(x)

    skips = reversed(skips[:-1])

    # Upsampling and establishing the skip connections
    for up, skip in zip(up_stack, skips):
      x = up(x)
      x = tf.keras.layers.Concatenate()([x, skip])

    x = last(x)

    return tf.keras.Model(inputs=inputs, outputs=x)

  def build_discriminator(self):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # source image input
    in_src_image = Input(shape=image_shape)
    # target image input
    in_target_image = Input(shape=image_shape)
    # concatenate images channel-wise
    merged = Concatenate()([in_src_image, in_target_image])
    # C64
    d = Conv3D(64, (3,3,3), strides=(2,2,2), padding='same', kernel_initializer=init)(merged)
    d = LeakyReLU(alpha=0.2)(d)
    # C128
    d = Conv3D(128, (3,3,3), strides=(2,2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # C256
    d = Conv3D(256, (3,3,3), strides=(2,2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # C512
    d = Conv3D(512, (3,3,3), strides=(2,2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    
    # second last output layer
    d = Conv3D(512, (3,3,3), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # patch output
    d = Conv3D(1, (3,3,3), padding='same', kernel_initializer=init)(d)
    patch_out = Activation('sigmoid')(d)
    # define model
    model = Model([in_src_image, in_target_image], patch_out)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])

    return model
'''

"\nclass GAN():\n  def __init__(self):\n    self.height = 64\n    self.width = 64\n    self.depth = 64\n    self.channel = 1\n    self.img_shape = (self.height, self.width, self.depth, self.channel)\n\n    optimizer = Adam(lr=0.002, beta_1=0.5, beta_2=0.999)\n\n    self.discriminator = self.build_discriminator()\n    self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])\n\n    self.generator = self.build_generator()\n    self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)\n\n    in_src = Input(shape=self.img_shape)\n    img = self.generator(in_src)\n\n    self.discriminator.trainable = False\n\n    valid = self.discriminator([in_src,img])\n  \n    self.combined = Model(in_src, [valid, img])\n    self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)\n    \n  def downsample(self, n_filters, size, apply_batchnorm=True):\n    initializer = tf.random_normal_initializer(0., 0.02)\n\n    result = tf.keras.Sequent

In [21]:
class GAN():
  def __init__(self, shape):
    self.shape = shape

    optimizer = Adam(lr=0.002, beta_1=0.5, beta_2=0.999)

    '''
    self.discriminator = self.define_discriminator()
    self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    self.generator = self.define_generator()
    self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)

    in_src = Input(shape=self.shape)
    img = self.generator(in_src)

    self.discriminator.trainable = False

    valid = self.discriminator([in_src,img])
  
    self.combined = Model(in_src, [valid, img])
    self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)
    '''


  def encoder_block(self,layer_in, n_filters, batchnorm=True):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # add downsampling layer
    g = Conv3D(n_filters, (3,3,3), strides=(2,2,2), padding='same', kernel_initializer=init)(layer_in)
    # conditionally add batch normalization
    if batchnorm:
      g = BatchNormalization()(g, training=True)
    # leaky relu activation
    g = LeakyReLU(alpha=0.2)(g)
    return g
  
  # define a decoder block
  def decoder_block(self,layer_in, skip_in, n_filters, dropout=True):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # add upsampling layer
    g = Conv3DTranspose(n_filters, (3,3,3), strides=(2,2,2), padding='same', kernel_initializer=init)(layer_in)
    # add batch normalization
    g = BatchNormalization()(g, training=True)
    # conditionally add dropout
    if dropout:
      g = Dropout(0.5)(g, training=True)
    # merge with skip connection
    g = Concatenate()([g, skip_in])
    # relu activation
    g = Activation('relu')(g)
    return g
  
  # define the standalone generator model
  def define_generator(self, condition):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # image input
    in_image = Input(shape=self.shape)
    

    aa = condition * tf.ones([self.shape[0],1])
    bb = tf.concat([in_image, aa])
    
    
    print("aa: ", aa.shape)

    # encoder model
    e1 = self.encoder_block(in_image, 64, batchnorm=False)
    e2 = self.encoder_block(e1, 128)
    e3 = self.encoder_block(e2, 256)
    e4 = self.encoder_block(e3, 512)
    e5 = self.encoder_block(e4, 512)
    
    # bottleneck, no batch norm and relu
    b = Conv3D(512, (4,4,4), strides=(2,2,2), padding='same', kernel_initializer=init)(e5)
    b = Activation('relu')(b)
    # decoder model
    d1 = self.decoder_block(b, e5, 512)
    d2 = self.decoder_block(d1, e4, 512)
    d3 = self.decoder_block(d2, e3, 256)
    d4 = self.decoder_block(d3, e2, 128, dropout=False)
    d5 = self.decoder_block(d4, e1, 64, dropout=False)

    # output
    g = Conv3DTranspose(1, (3,3,3), strides=(2,2,2), padding='same', kernel_initializer=init)(d5)
    out_image = Activation('tanh')(g)
    # define model
    model = Model(in_image, out_image)

    return model

  def define_discriminator(self):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # source image input
    in_src_image = Input(shape=self.shape)
    # target image input
    in_target_image = Input(shape=self.shape)
    # concatenate images channel-wise
    merged = Concatenate()([in_src_image, in_target_image])
    # C64
    d = Conv3D(64, (3,3,3), strides=(2,2,2), padding='same', kernel_initializer=init)(merged)
    d = LeakyReLU(alpha=0.2)(d)
    # C128
    d = Conv3D(128, (3,3,3), strides=(2,2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # C256
    d = Conv3D(256, (3,3,3), strides=(2,2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # C512
    d = Conv3D(512, (3,3,3), strides=(2,2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
      
    # second last output layer
    d = Conv3D(512, (3,3,3), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # patch output
    d = Conv3D(1, (3,3,3), padding='same', kernel_initializer=init)(d)
    patch_out = Activation('sigmoid')(d)
    # define model
    model = Model([in_src_image, in_target_image], patch_out)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])

    return model

  #define the combined generator and discriminator model, for updating the generator
  def define_gan(self, g_model, d_model):
    # make weights in the discriminator not trainable
    for layer in d_model.layers:
      if not isinstance(layer, BatchNormalization):
        layer.trainable = False
    # define the source image
    in_src = Input(shape=self.shape)
    # connect the source image to the generator input
    gen_out = g_model(in_src)
    # connect the source input and generator output to the discriminator input
    dis_out = d_model([in_src, gen_out])
    # src image as input, generated image and classification output
    model = Model(in_src, [dis_out, gen_out])
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss=['binary_crossentropy', 'mae'], optimizer=opt, loss_weights=[1,100])
    return model

  
  gan = GAN((64,64,64,1))
  a = gan.define_generator(1.0)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


ValueError: ignored

In [4]:
class DataLoader():

  def __init__(self, o_path, t_path):

    self.o_dataset = []
    self.t_dataset = []
    self.org_list = []
    self.tar_list = []
    self.o_path = o_path
    self.t_path = t_path
    self.filename = 'dataset.npz'


  def preprocess(self):
    
    fig = plt.figure(figsize=(10,30))

    for org_file in glob.glob(self.o_path):
      org_base = os.path.basename(org_file)
      org_base = org_base[0:15]
      for tar_file in glob.glob(self.t_path):
          tar_base = os.path.basename(tar_file)
          tar_base = tar_base[0:15]

          if tar_base == org_base:
            org_img = nib.load(org_file)
            tar_img = nib.load(tar_file)

            # convert nii file to numpy array
            data1 = org_img.get_fdata()
            data2 = tar_img.get_fdata()

            self.org_list.append(data1)
            self.tar_list.append(data2)

    num = 1

    # patchify and convert to tensor
    for o_data in self.org_list:
      src_patches = patchify(o_data, (64,64,64), step = 32)
      
      
      ax1 = fig.add_subplot(len(self.org_list),2,num)
      ax1.set_title('Base-Level')
      ax1.imshow(src_patches[1,2,3,:,:,40])
      num += 2
      

      # combine vectors 4 x 5 x 4 to 80 patches
      input_img = np.reshape(src_patches, (-1, src_patches.shape[3], src_patches.shape[4], src_patches.shape[5], 1)) # n_patches, x, y, z
      input_img = tf.convert_to_tensor(input_img, np.float32)
      self.o_dataset.append(input_img)

    num = 2

    # patchify and convert to tensor
    for t_data in self.tar_list:
      target_patches = patchify(t_data, (64,64,64), step = 32)  
      
      ax2 = fig.add_subplot(len(self.tar_list),2,num)
      ax2.set_title('Target')
      ax2.imshow(target_patches[1,2,3,:,:,40])
      num += 2
      
      # combine vectors 4 x 5 x 4 to 80 patches
      input_img = np.reshape(src_patches, (-1, src_patches.shape[3], src_patches.shape[4], src_patches.shape[5], 1)) # n_patches, x, y, z
      input_img = tf.convert_to_tensor(input_img, np.float32)
      self.t_dataset.append(input_img)

    return self.o_dataset, self.t_dataset


  def compress(self, a, b):
    savez_compressed(self.filename, a, b)


  # load and prepare training images
  def load_real_samples(self):
    # load compressed arrays
    data = load(self.filename)
    # unpack arrays
    X1, X2 = data['arr_0'], data['arr_1']
    return [X1, X2]


In [11]:
# Create Models
gan = GAN((64,64,64,1))
a = gan.define_generator()
b = gan.define_discriminator()
model = gan.define_gan(a,b)

model.summary()
tf.keras.utils.plot_model(model, show_shapes=True, show_layer_names=True)

#tf.keras.utils.plot_model(a, show_shapes=True, show_layer_names=True)
#tf.keras.utils.plot_model(b, show_shapes=True, show_layer_names=True)


'''
# Preprocess MRI images
data = DataLoader('/content/drive/MyDrive/Data/Original/*.gz', '/content/drive/MyDrive/Data/Target/*.gz')
a, b = data.preprocess()

data.compress(a,b)
dataset = data.load_real_samples()
print(dataset[1].shape)
'''

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model: "model_24"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_34 (InputLayer)           [(None, 64, 64, 64,  0                                            
__________________________________________________________________________________________________
model_22 (Functional)           (None, 64, 64, 64, 1 59043521    input_34[0][0]                   
__________________________________________________________________________________________________
model_23 (Functional)           (None, 4, 4, 4, 1)   11747137    input_34[0][0]                   
                                                                 model_22[0][0]                   
Total params: 70,790,658
Trainable params: 59,040,577
Non-trainable params: 11,750,081
__________________________________________________________________________________________________


"\n# Preprocess MRI images\ndata = DataLoader('/content/drive/MyDrive/Data/Original/*.gz', '/content/drive/MyDrive/Data/Target/*.gz')\na, b = data.preprocess()\n\ndata.compress(a,b)\ndataset = data.load_real_samples()\nprint(dataset[1].shape)\n"